# MLlib package of PySpark
- spark 2.x 부터 ML 패키지는 데이터프레임에 대해 작동
    - MLlib 패키지는 RDD에 대해 작동함
    
- MLlib은 전체적으로 크게 세단계의 머신러닝 기능으로 구분된다
    1. 데이터 전처리 : 
        - 피처추출, 변형, 선택, 카테고리 피처에 대한 해석, 자연어처리
    1. 머신러닝 알고리즘 :
        - 몇몇 유명, 고급레벨인 회귀, 분류, 군집 알고리즘 지원
    1. 유틸리티 :
        - 기술통계, 카이스퀘어(ChiSquare)테스트, 선형대수, 모델평가
        
- https://books.google.co.kr/books?id=HVQoDwAAQBAJ&pg=PA84&lpg=PA84&dq=births_transformed+%3D+births_transformed.select(exprs_YNU)&source=bl&ots=tLNsHpKdfH&sig=ACfU3U37yj0SiQWLB-DgXvOuT1toOs4keQ&hl=ko&sa=X&ved=2ahUKEwjissX9v5HpAhWS7WEKHbfMAkIQ6AEwAHoECAoQAQ#v=onepage&q&f=false

In [5]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf  = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc    = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

### INFANT_ALIVE_AT_REPORT 가 1인지, 0 인지를 예측 하는것이 목표
- US 2014 ~ 2015 년 출생데이터의 일부
    - 원본데이터는 300개의 피쳐로 구성, 그중 85개 선별

- 799 만개의 데이터 중 45429개 균등 샘플링

In [1]:
import pyspark.sql.types as typ

In [2]:
labels = [
    ('INFANT_ALIVE_AT_REPORT', typ.StringType()),
    ('BIRTH_YEAR', typ.IntegerType()),
    ('BIRTH_MONTH', typ.IntegerType()),
    ('BIRTH_PLACE', typ.StringType()),
    ('MOTHER_AGE_YEARS', typ.IntegerType()),
    ('MOTHER_RACE_6CODE', typ.StringType()),
    ('MOTHER_EDUCATION', typ.StringType()),
    ('FATHER_COMBINED_AGE', typ.IntegerType()),
    ('FATHER_EDUCATION', typ.StringType()),
    ('MONTH_PRECARE_RECODE', typ.StringType()),
    ('CIG_BEFORE', typ.IntegerType()),
    ('CIG_1_TRI', typ.IntegerType()),
    ('CIG_2_TRI', typ.IntegerType()),
    ('CIG_3_TRI', typ.IntegerType()),
    ('MOTHER_HEIGHT_IN', typ.IntegerType()),
    ('MOTHER_BMI_RECODE', typ.IntegerType()),
    ('MOTHER_PRE_WEIGHT', typ.IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', typ.IntegerType()),
    ('MOTHER_WEIGHT_GAIN', typ.IntegerType()),
    ('DIABETES_PRE', typ.StringType()),
    ('DIABETES_GEST', typ.StringType()),
    ('HYP_TENS_PRE', typ.StringType()),
    ('HYP_TENS_GEST', typ.StringType()),
    ('PREV_BIRTH_PRETERM', typ.StringType()),
    ('NO_RISK', typ.StringType()),
    ('NO_INFECTIONS_REPORTED', typ.StringType()),
    ('LABOR_IND', typ.StringType()),
    ('LABOR_AUGM', typ.StringType()),
    ('STEROIDS', typ.StringType()),
    ('ANTIBIOTICS', typ.StringType()),
    ('ANESTHESIA', typ.StringType()),
    ('DELIV_METHOD_RECODE_COMB', typ.StringType()),
    ('ATTENDANT_BIRTH', typ.StringType()),
    ('APGAR_5', typ.IntegerType()),
    ('APGAR_5_RECODE', typ.StringType()),
    ('APGAR_10', typ.IntegerType()),
    ('APGAR_10_RECODE', typ.StringType()),
    ('INFANT_SEX', typ.StringType()),
    ('OBSTETRIC_GESTATION_WEEKS', typ.IntegerType()),
    ('INFANT_WEIGHT_GRAMS', typ.IntegerType()),
    ('INFANT_ASSIST_VENTI', typ.StringType()),
    ('INFANT_ASSIST_VENTI_6HRS', typ.StringType()),
    ('INFANT_NICU_ADMISSION', typ.StringType()),
    ('INFANT_SURFACANT', typ.StringType()),
    ('INFANT_ANTIBIOTICS', typ.StringType()),
    ('INFANT_SEIZURES', typ.StringType()),
    ('INFANT_NO_ABNORMALITIES', typ.StringType()),
    ('INFANT_ANCEPHALY', typ.StringType()),
    ('INFANT_MENINGOMYELOCELE', typ.StringType()),
    ('INFANT_LIMB_REDUCTION', typ.StringType()),
    ('INFANT_DOWN_SYNDROME', typ.StringType()),
    ('INFANT_SUSPECTED_CHROMOSOMAL_DISORDER', typ.StringType()),
    ('INFANT_NO_CONGENITAL_ANOMALIES_CHECKED', typ.StringType()),
    ('INFANT_BREASTFED', typ.StringType())
]

In [3]:
schema = typ.StructType([ typ.StructField(e[0], e[1], False)
                        for e in labels ])

In [11]:
base_path = "../../data/RDD_example/"

In [12]:
births = spark.read.csv(base_path+'births_train.csv', header=True, schema=schema)

In [13]:
type(births)

pyspark.sql.dataframe.DataFrame

In [14]:
births.printSchema()

root
 |-- INFANT_ALIVE_AT_REPORT: string (nullable = true)
 |-- BIRTH_YEAR: integer (nullable = true)
 |-- BIRTH_MONTH: integer (nullable = true)
 |-- BIRTH_PLACE: string (nullable = true)
 |-- MOTHER_AGE_YEARS: integer (nullable = true)
 |-- MOTHER_RACE_6CODE: string (nullable = true)
 |-- MOTHER_EDUCATION: string (nullable = true)
 |-- FATHER_COMBINED_AGE: integer (nullable = true)
 |-- FATHER_EDUCATION: string (nullable = true)
 |-- MONTH_PRECARE_RECODE: string (nullable = true)
 |-- CIG_BEFORE: integer (nullable = true)
 |-- CIG_1_TRI: integer (nullable = true)
 |-- CIG_2_TRI: integer (nullable = true)
 |-- CIG_3_TRI: integer (nullable = true)
 |-- MOTHER_HEIGHT_IN: integer (nullable = true)
 |-- MOTHER_BMI_RECODE: integer (nullable = true)
 |-- MOTHER_PRE_WEIGHT: integer (nullable = true)
 |-- MOTHER_DELIVERY_WEIGHT: integer (nullable = true)
 |-- MOTHER_WEIGHT_GAIN: integer (nullable = true)
 |-- DIABETES_PRE: string (nullable = true)
 |-- DIABETES_GEST: string (nullable = true)


- Y : yes
- N : no
- U : unknown

In [15]:
recode_dictionary = {
    'YNU' : {
         'Y' : 1
        ,'N' : 2
        ,'U' : 0
    }
}

In [16]:
type(recode_dictionary), len(recode_dictionary)

(dict, 1)

In [9]:
selected_features = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_BEFORE', 
    'CIG_1_TRI', 
    'CIG_2_TRI', 
    'CIG_3_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'MOTHER_DELIVERY_WEIGHT', 
    'MOTHER_WEIGHT_GAIN', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]

In [17]:
births_trimmed = births.select(selected_features)

In [18]:
births_trimmed.printSchema()

root
 |-- INFANT_ALIVE_AT_REPORT: string (nullable = true)
 |-- BIRTH_PLACE: string (nullable = true)
 |-- MOTHER_AGE_YEARS: integer (nullable = true)
 |-- FATHER_COMBINED_AGE: integer (nullable = true)
 |-- CIG_BEFORE: integer (nullable = true)
 |-- CIG_1_TRI: integer (nullable = true)
 |-- CIG_2_TRI: integer (nullable = true)
 |-- CIG_3_TRI: integer (nullable = true)
 |-- MOTHER_HEIGHT_IN: integer (nullable = true)
 |-- MOTHER_PRE_WEIGHT: integer (nullable = true)
 |-- MOTHER_DELIVERY_WEIGHT: integer (nullable = true)
 |-- MOTHER_WEIGHT_GAIN: integer (nullable = true)
 |-- DIABETES_PRE: string (nullable = true)
 |-- DIABETES_GEST: string (nullable = true)
 |-- HYP_TENS_PRE: string (nullable = true)
 |-- HYP_TENS_GEST: string (nullable = true)
 |-- PREV_BIRTH_PRETERM: string (nullable = true)




- 데이터셋에는 YES, NO, UNKNOWN 값을 가진 피쳐들이 매우 많음
    - YES만 1, 나머지는 0으로 변환
- 어미의 흡연량 관련 레코드 :
    - 0      : 임신기간 동안 금연
    - 1 ~ 97 : 1 ~ 97 개피 흡연
    - 98     : 98개피 이상
    - 99     : 알수없음

In [19]:
import pyspark.sql.functions as func

def recode(col, key):
    return recode_dictionary[key][col]

- 99 : 알수없음을 제외한 것은 그대로 반환하고, 99 일때는 0을 반환

In [20]:
def correct_cig(feat):
    return func.when(func.col(feat) != 99, func.col(feat)).otherwise(0)

## 사용자 정의 함수 써보기
- def 으로 정의한것을 넣어서

In [21]:
rec_integer = func.udf( recode, typ.IntegerType())

In [22]:
births_transformed = births_trimmed \
                    .withColumn('CIG_BEFORE', correct_cig('CIG_BEFORE')) \
                    .withColumn('CIG_1_TRI', correct_cig('CIG_1_TRI')) \
                    .withColumn('CIG_2_TRI', correct_cig('CIG_2_TRI')) \
                    .withColumn('CIG_3_TRI', correct_cig('CIG_3_TRI'))

- births_trimmed.schema 에는 이름, 데이터타입, nullable 여부가 있는데
- 이중 name, dataType만 가져오겠다

In [23]:
cols = [ (col.name, col.dataType) for col in births_trimmed.schema ]
cols

[('INFANT_ALIVE_AT_REPORT', StringType),
 ('BIRTH_PLACE', StringType),
 ('MOTHER_AGE_YEARS', IntegerType),
 ('FATHER_COMBINED_AGE', IntegerType),
 ('CIG_BEFORE', IntegerType),
 ('CIG_1_TRI', IntegerType),
 ('CIG_2_TRI', IntegerType),
 ('CIG_3_TRI', IntegerType),
 ('MOTHER_HEIGHT_IN', IntegerType),
 ('MOTHER_PRE_WEIGHT', IntegerType),
 ('MOTHER_DELIVERY_WEIGHT', IntegerType),
 ('MOTHER_WEIGHT_GAIN', IntegerType),
 ('DIABETES_PRE', StringType),
 ('DIABETES_GEST', StringType),
 ('HYP_TENS_PRE', StringType),
 ('HYP_TENS_GEST', StringType),
 ('PREV_BIRTH_PRETERM', StringType)]

- Y or N 을 찾기
    - StringType이겠네?
    

In [24]:
YNU_cols = []
for i, s in enumerate(cols):
    if s[1] == typ.StringType():
        dis = births.select(s[0]) \
        .distinct() \
        .rdd \
        .map(lambda row: row[0]) \
        .collect()
        print(dis)
        if 'Y' in dis:
            YNU_cols.append(s[0])
            print(YNU_cols)

['Y', 'N']
['INFANT_ALIVE_AT_REPORT']
['7', '3', '5', '6', '9', '1', '4', '2']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST', 'HYP_TENS_PRE']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST', 'HYP_TENS_PRE', 'HYP_TENS_GEST']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST', 'HYP_TENS_PRE', 'HYP_TENS_GEST', 'PREV_BIRTH_PRETERM']


['Y', 'N']
['INFANT_ALIVE_AT_REPORT']
['7', '3', '5', '6', '9', '1', '4', '2']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST', 'HYP_TENS_PRE']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST', 'HYP_TENS_PRE', 'HYP_TENS_GEST']
['Y', 'U', 'N']
['INFANT_ALIVE_AT_REPORT', 'DIABETES_PRE', 'DIABETES_GEST', 'HYP_TENS_PRE', 'HYP_TENS_GEST', 'PREV_BIRTH_PRETERM']

In [25]:
births.select('INFANT_ALIVE_AT_REPORT') \
        .distinct() \
        .rdd \
        .map(lambda row: row[0]) \
        .collect()

['Y', 'N']

In [26]:
births.select('BIRTH_PLACE') \
        .distinct() \
        .rdd \
        .map(lambda row: row[0]) \
        .collect()

['7', '3', '5', '6', '9', '1', '4', '2']

In [27]:
births.select([
    'INFANT_NICU_ADMISSION',
    rec_integer(
        'INFANT_NICU_ADMISSION'
        , func.lit('YNU')).alias('INFANT_NICU_ADMISSION_RECODE')
]).take(5)

[Row(INFANT_NICU_ADMISSION='Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION='Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION='U', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION='N', INFANT_NICU_ADMISSION_RECODE=2),
 Row(INFANT_NICU_ADMISSION='U', INFANT_NICU_ADMISSION_RECODE=0)]

In [28]:
exprs_YNU = [
    rec_integer(x, func.lit('YNU')).alias(x)
    if x in YNU_cols
    else x
    for x in births_transformed.columns
]

In [29]:
exprs_YNU

[Column<b'recode(INFANT_ALIVE_AT_REPORT, YNU) AS `INFANT_ALIVE_AT_REPORT`'>,
 'BIRTH_PLACE',
 'MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_BEFORE',
 'CIG_1_TRI',
 'CIG_2_TRI',
 'CIG_3_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'MOTHER_DELIVERY_WEIGHT',
 'MOTHER_WEIGHT_GAIN',
 Column<b'recode(DIABETES_PRE, YNU) AS `DIABETES_PRE`'>,
 Column<b'recode(DIABETES_GEST, YNU) AS `DIABETES_GEST`'>,
 Column<b'recode(HYP_TENS_PRE, YNU) AS `HYP_TENS_PRE`'>,
 Column<b'recode(HYP_TENS_GEST, YNU) AS `HYP_TENS_GEST`'>,
 Column<b'recode(PREV_BIRTH_PRETERM, YNU) AS `PREV_BIRTH_PRETERM`'>]

- YNU_cols에 있으면 사용자정의함수 처리 되고 아니면 그냥 컬럼명이 리스트에 담김

In [ ]:
births_transformed = births_transformed.select(exprs_YNU)

In [ ]:
births_transformed.select(YNU_cols[-5:]).show(5)

In [ ]:
YNU_cols[-5:]

In [ ]:
# Descriptive statistics

import pyspark.mllib.stat as st
import numpy as np

In [ ]:
numeric_cols = [
#     'INFANT_ALIVE_AT_REPORT', 
#     'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_BEFORE', 
    'CIG_1_TRI', 
    'CIG_2_TRI', 
    'CIG_3_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'MOTHER_DELIVERY_WEIGHT', 
    'MOTHER_WEIGHT_GAIN', 
]

In [ ]:
numeric_rdd = births_transformed.select(numeric_cols) \
                .rdd.map(lambda row: [e for e in row])

In [ ]:
mllib_stats = st.Statistics.colStats(numeric_rdd)

In [ ]:
for col, m, v in zip(numeric_cols, mllib_stats.mean(), mllib_stats.variance()):
    print( '{0:.10s}: \t{1:.2f} \t{2:.2f}'.format(col, m, np.sqrt(v)) )

- colStats()
    - 처리된 값은 기술통계를 샘플링해서 계산한 것임
    - 실제 데이터에서는 큰 문제는 없다. 
    - 데이터셋이 100개 미만일 경우에는 문제가 될 수 있다
    - 본 함수는 RDD 데이터를 취해 기술통계를 계산한다.
        - MulhvariateStahshcalSummary 객체를 리턴
            - count() : 데이터행 갯수
            - max() : 최대값
            - numNonzeros() : 0 이 아닌 갯수
            - variance() : 분산
                - Sample variance vector.
            - norml1() : L1-Norm 값
                - L1 norm of each column
            - norml2() : L2-Norm 값
                - Euclidean magnitude of each column

                - L1 Loss 이냐 L2 Loss 이냐 선택?
                    - 직관적으로 오차의 제곱을 더함
                    - Outlier에 대하여 더 Robust하다
                    - Outlier가 적당히 무시되길 원하면 L1 Loss를
                    - Outlier에 신경을 써야한다면 L2 Loss를 사용

In [ ]:
categorical_cols = [e for e in births_transformed.columns if e not in numeric_cols]

In [ ]:
categorical_rdd = births_transformed.select(categorical_cols) \
.rdd.map(lambda row: [e for e in row])

In [ ]:
for i, col in enumerate(categorical_cols):
    agg = categorical_rdd.groupBy(lambda row: row[i]) \
                        .map( lambda row : (row[0], len(row[1])) )
    print( col, sorted(agg.collect(), key=lambda el: el[1], reverse=True) )

In [ ]:
corrs = st.Statistics.corr(numeric_rdd)

In [ ]:
for i, el in enumerate(corrs > 0.5):
    correlated = [
        (numeric_cols[j], corrs[i][j])
        for j, e in enumerate(el)
        if e == 1.0 and j != i
    ]
    if len(correlated) > 0:
        for e in correlated:
            print( '{0}-to-{1}: {2:.2f}' \
                 .format(numeric_cols[i], e[0], e[1]) )

In [ ]:
births_transformed.columns

In [ ]:
features_to_keep = [
 'INFANT_ALIVE_AT_REPORT',
 'BIRTH_PLACE',
 'MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_1_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'DIABETES_PRE',
 'DIABETES_GEST',
 'HYP_TENS_PRE',
 'HYP_TENS_GEST',
 'PREV_BIRTH_PRETERM'
]

In [ ]:
births_transformed = births_transformed.select([e for e in features_to_keep])

In [ ]:
import pyspark.mllib.linalg as ln

In [ ]:
for cat in categorical_cols[1:]:
    agg = births_transformed.groupby('INFANT_ALIVE_AT_REPORT') \
                            .pivot(cat).count()
    agg_rdd = agg.rdd.map(lambda row: (row[1:])) \
    .flatMap(lambda row: [0 if e == None else e for e in row]) \
    .collect()
    row_length = len(agg.collect()[0])-1
    agg = ln.Matrices.dense(row_length, 2, agg_rdd)
    
    test = st.Statistics.chiSqTest(agg)
    print(cat, round(test.pValue, 4))

- Matrices.dense() 함수의 파라미터 설명
    1. 행렬에서 행의 수
    1. 카테고리 고유값의 수
    1. 행렬로 변환 할 값들에 대한 리스트

### final dataset


In [ ]:
import pyspark.mllib.feature as ft
import pyspark.mllib.regression as reg

hashing = ft.HashingTF(7)
births_hashed = births_transformed.rdd.map( lambda row: [
                list(hashing.transform(row[1]).toArray())
                    if col == 'BIRTH_PLACE'
                    else row[i] 
                for i, col in enumerate(features_to_keep)] 
                                         ) \
    .map( lambda row: [[e] if type(e) == int else e for e in row] ) \
    .map( lambda row: [item for sublist in row for item in sublist] ) \
    .map( lambda row: reg.LabeledPoint(row[0], ln.Vectors.dense(row[1:])) )

### train and test data 

In [ ]:
births_train, births_test = births_hashed.randomSplit([0.6, 0.4])

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
LR_Model = LogisticRegressionWithLBFGS.train(births_train, iterations=10)

In [ ]:
LR_results = (
    births_test.map(lambda row: row.label) \
    .zip( LR_Model.predict(births_test.map(lambda row: row.features)) )
).map( lambda row: (row[0], row[1] * 1.0) )

In [ ]:
LR_results.collect()


### Model Evaluation

In [ ]:
import pyspark.mllib.eveluation as ev

In [ ]:
LR_evaluation = ev.BinaryClassificationMetrics(LR_results)

print( 'Area under PR: {0:.2f}'.format(LR_evaluation.areaUnderPR) )
print( 'Araed under ROC: {0:.2f}'.format(LR_evaluation.areaUnderROC) )
LR_evaluation.unpersist()

## ChiSqSelector()
- 피쳐 선택

In [ ]:
selector = ft.ChiSqSelector(4).fit(births_train)

In [ ]:
topFeatures_train = (
    births_train.map(lambda row: row.label)\
            .zip(selector.transform(births_train.map(lambda row: row.features)))
).map(lambda row: reg.LabeledPoint(row[0], row[1]))

In [ ]:
topFeatures_test = (
births_test.map(lambda row: row.label)\
.zip(selector.transform(births_test.map(lambda row: row.features)))
).map(lambda row: reg.LabeledPoint(row[0], row[1]))

## Random Forest
- trainClassifier  
    - 첫번째 : 트레인데이타  
    - 두번째 : 타깃이 몇개인지  
    - 세번째 : 딕셔너리타입 (키 : 학습데이터셋RDD의 카테고리 피처의 인덱스, 값: 각 카테고리 피처가 가지고있는 레벨 수  
    - 네번째 : 랜덤 포레스트 모델에서 생성할 트리 수  

In [ ]:
from pyspark.mllib.tree import RandomForest
RF_model = RandomForest.trainClassifier(data=topFeatures_train,
                                       numClasses=2,
                                       categoricalFeaturesInfo={},
                                       numTrees=6,
                                       featureSubsetStrategy='all',
                                       seed=5)

In [ ]:
# 모델 성능 평가
RF_results = (
    topFeatures_test.map(lambda row: row.label)\
                .zip(RF_model.predict(topFeatures_test.map(lambda row: row.features)))
)

In [ ]:
RF_evaluation = ev.BinaryClassificationMetrics(RF_results)

print('Area under PR: {0:.2f}'.format(RF_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}'.format(RF_evaluation.areaUnderROC))
RF_evaluation.unpersist()